In [ ]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from pprint import pprint
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import activity_classifier as ac

FP_ANNOT_CONSISTENCY_OUT = '/Users/bryanfry/code/activity-classifier/activity_classifier/EDA_output/EDA_02_annot_consistency.csv'

FP_METADATA = f'{ac.CAP24_DIR}/metadata.csv'
FP_ANNOT_DICT = f'{ac.CAP24_DIR}/annotation-label-dictionary.csv'
FPS_GZ_DATA = sorted([f'{ac.CAP24_DIR}/{f}' for f in os.listdir(ac.CAP24_DIR) if f.endswith(".csv.gz")])

In [ ]:
def annot_consistency(df, fn, chunk_duration=10.):
    ss = [] # to populate with pd.Series elements, each describing annot consistencuy for one chunk
    ck = ac.df_to_chunkable_ts(df)
    cs = ck.get_chunks_by_time(chunk_duration)
    for c in cs:
        annot = c.annot[:, 0]
        #vc = pd.value_counts(annot, dropna=False)
        vc = pd.Series(annot).value_counts(dropna=False)
        s = pd.Series(dict(
            file = fn,
            t0 = c.t0,
            n = c.n,
            n_missing = sum(pd.isnull(annot)),
            annot_counts = json.dumps(vc.to_dict()),
            n_class = len(vc)
        ))
        ss.append(s)
    return pd.DataFrame(ss)


dfas = []   # To populate with pd.DataFrame elements, each describing annot
            # consistency for one file
for i, fp in enumerate(FPS_GZ_DATA):
    print('Processing file: ', fp, '(', i+1, 'of', len(FPS_GZ_DATA), ')')
    df = ac.load_acc_data_file(fp)
    dfa = annot_consistency(df, os.path.basename(fp), chunk_duration=10.)
    dfas.append(dfa)

dfas = pd.concat(dfas, ignore_index=True)
dfas.to_csv(FP_ANNOT_CONSISTENCY_OUT, index=False)

In [ ]:
dfas['n_missing'].value_counts()

,file,t0,n,n_missing,annot_counts,n_class
0,P001.csv.gz,0.0,1000,0,"{""7030 sleeping;MET 0.95"": 1000}",1
1,P001.csv.gz,10.0,1000,0,"{""7030 sleeping;MET 0.95"": 1000}",1
2,P001.csv.gz,20.0,1000,0,"{""7030 sleeping;MET 0.95"": 1000}",1
3,P001.csv.gz,30.0,1000,0,"{""7030 sleeping;MET 0.95"": 1000}",1
4,P001.csv.gz,40.0,1000,0,"{""7030 sleeping;MET 0.95"": 1000}",1
...,...,...,...,...,...,...
1398029,P151.csv.gz,107650.0,1000,0,"{""7030 sleeping;MET 0.95"": 1000}",1
1398030,P151.csv.gz,107660.0,1000,0,"{""7030 sleeping;MET 0.95"": 1000}",1
1398031,P151.csv.gz,107670.0,1000,0,"{""7030 sleeping;MET 0.95"": 1000}",1
1398032,P151.csv.gz,107680.0,1000,0,"{""7030 sleeping;MET 0.95"": 1000}",1
